<a href="https://colab.research.google.com/github/johnny7861532/DCGAN/blob/master/Copy_of_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir results

In [0]:
cuda_enable = True

In [0]:
import torch
x = torch.cuda.FloatTensor(1)
x.get_device()
print (torch.cuda.is_available())
print (torch.cuda.device_count())

True
1


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torchvision.utils as visutils
from torch.autograd import Variable

batch_size = 64
image_size = 64

transform = transforms.Compose([transforms.Scale((image_size,image_size)),transforms.ToTensor()
,transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),])

imgdataset = dataset.CIFAR10(root = './data1',download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(imgdataset,batch_size = batch_size,shuffle = True
                                         ,num_workers = 2)

#cake_data = dataset.ImageFolder('./data', transform=transform)
#cake_dataloader = torch.utils.data.DataLoader(cake_data,batch_size = batch_size,shuffle = True
#                                         ,num_workers = 2)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0,0.02)
    elif classname.find('BatchNorm') != -1:
        #orginal setting is (0.0,0.02) but (1.0.0.002) result is much better
        m.weight.data.normal_(1.0,0.02)
        m.bias.data.fill_(0)

class G(nn.Module):
    #when u start a class u will always need to init func
    def __init__(self):
        super(G,self).__init__()
        self.main = nn.Sequential(
                #100 is input tensor , 512 is output tensor, 4 is kernel size
                #1 is stride and 0 is padding
                #keras using Conv2DTranspose
                nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
                nn.BatchNorm2d(512), 
                nn.ReLU(True),
                nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
                nn.BatchNorm2d(256),
                nn.ReLU(True),
                nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
                nn.BatchNorm2d(128),
                nn.ReLU(True),
                nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
                nn.Tanh()
                 )
    # forward input data through ooutput
    def forward(self,input):
        output = self.main(input)
        return output
    
#inital generator
netG = G().cuda()
netG.apply(weights_init)

class D(nn.Module):
    def __init__(self):
        super(D,self).__init__()
        self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1,bias = False),
                nn.LeakyReLU(0.2,inplace = True),
                nn.Conv2d(64, 128, 4, 2, 1,bias = False),
                nn.LeakyReLU(0.2,inplace = True),
                nn.Conv2d(128, 256, 4, 2, 1,bias = False),
                nn.LeakyReLU(0.2,inplace = True),
                nn.Conv2d(256, 512, 4, 2, 1,bias = False),
                nn.LeakyReLU(0.2,inplace = True),
                nn.Conv2d(512,1,4,1,0, bias = False),
                nn.Sigmoid()
                
                )
    def forward(self,input):
        output = self.main(input)
        return output.view(-1)
        
netD = D().cuda()
netD.apply(weights_init)   


#training DCGAN
#BCE == Binary cross entropy
criterion = nn.BCELoss()

optD = optim.Adam(netD.parameters(),lr = 0.00004,betas = (0.5,0.999))
optG = optim.Adam(netG.parameters(),lr = 0.0002,betas = (0.5,0.999))

#set up epochs
for epoch in range(1000):
    for i, data in enumerate(dataloader ,0):
        #first step: update the weight of NN of D
        netD.zero_grad()
        #second step: training the D with real image
        real, _ = data
        #pytorch only take torch variable so we need to using varible first
        input  = Variable(real)
        #because this is the real image that we need to tag this one as 1 
        #represent the true image for training discrimiator
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input.cuda())
        #using crterion to cuculate the err from output and target
        errD_real = criterion(output,target.cuda())
        #third step: training the D with fake image
        # we create an nosie for generating image, and also this parameter 
        # must be torch.Variable aslo
        noise = Variable(torch.randn(input.size()[0],100,1,1))
        #init the generate net
        fake = netG(noise.cuda())
        #mark the fake image as 0
        target = Variable(torch.zeros(input.size()[0]))
        #this just for the classifier so we don't update the netG, so we add
        #detach()
        output = netD(fake.detach())
        errD_fake = criterion(output,target.cuda())
        #backpropagation of the toal loss
        # sum up totall error
        errD = errD_real + errD_fake
        #backward the error
        errD.backward()
        #apply SDG (adam)
        optD.step()
        
        #set up the generator of nn
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        #we need to update the weight so we don't need detach
        output = netD(fake)
        #calculate the error
        errG = criterion(output,target.cuda())
        errG.backward()
        #update the weight
        optG.step()
        
        #print the loss and the generate image
        
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 1000, i, len(dataloader), errD.item(), errG.item()))
        if i % 100 == 0:
            visutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise.cuda())
            visutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
0it [00:00, ?it/s]

 99%|█████████▉| 169533440/170498071 [00:37<00:00, 5084802.57it/s]

[0/1000][0/782] Loss_D: 1.3863 Loss_G: 0.6930
[0/1000][1/782] Loss_D: 1.3911 Loss_G: 0.6763
[0/1000][2/782] Loss_D: 1.3973 Loss_G: 0.6639
[0/1000][3/782] Loss_D: 1.4037 Loss_G: 0.6536
[0/1000][4/782] Loss_D: 1.4015 Loss_G: 0.6466
[0/1000][5/782] Loss_D: 1.4002 Loss_G: 0.6419
[0/1000][6/782] Loss_D: 1.3974 Loss_G: 0.6397
[0/1000][7/782] Loss_D: 1.3907 Loss_G: 0.6377
[0/1000][8/782] Loss_D: 1.3857 Loss_G: 0.6346
[0/1000][9/782] Loss_D: 1.3878 Loss_G: 0.6311
[0/1000][10/782] Loss_D: 1.3820 Loss_G: 0.6283
[0/1000][11/782] Loss_D: 1.3760 Loss_G: 0.6253
[0/1000][12/782] Loss_D: 1.3763 Loss_G: 0.6253
[0/1000][13/782] Loss_D: 1.3743 Loss_G: 0.6265
[0/1000][14/782] Loss_D: 1.3607 Loss_G: 0.6271
[0/1000][15/782] Loss_D: 1.3431 Loss_G: 0.6281
[0/1000][16/782] Loss_D: 1.3276 Loss_G: 0.6292
[0/1000][17/782] Loss_D: 1.3124 Loss_G: 0.6291
[0/1000][18/782] Loss_D: 1.3101 Loss_G: 0.6286
[0/1000][19/782] Loss_D: 1.2971 Loss_G: 0.6271
[0/1000][20/782] Loss_D: 1.2889 Loss_G: 0.6243
[0/1000][21/782] Loss_D

170500096it [00:50, 5084802.57it/s]                               

[0/1000][46/782] Loss_D: 1.2064 Loss_G: 0.8099
[0/1000][47/782] Loss_D: 1.1964 Loss_G: 0.8123
[0/1000][48/782] Loss_D: 1.1602 Loss_G: 0.8151
[0/1000][49/782] Loss_D: 1.1366 Loss_G: 0.8138
[0/1000][50/782] Loss_D: 1.0775 Loss_G: 0.8046
[0/1000][51/782] Loss_D: 1.1066 Loss_G: 0.7760
[0/1000][52/782] Loss_D: 1.1657 Loss_G: 0.7190
[0/1000][53/782] Loss_D: 1.1754 Loss_G: 0.6537
[0/1000][54/782] Loss_D: 1.2438 Loss_G: 0.6031
[0/1000][55/782] Loss_D: 1.4000 Loss_G: 0.5716
[0/1000][56/782] Loss_D: 1.3451 Loss_G: 0.5642
[0/1000][57/782] Loss_D: 1.4286 Loss_G: 0.5685
[0/1000][58/782] Loss_D: 1.3015 Loss_G: 0.5798
[0/1000][59/782] Loss_D: 1.3305 Loss_G: 0.5880
[0/1000][60/782] Loss_D: 1.2865 Loss_G: 0.5873
[0/1000][61/782] Loss_D: 1.3244 Loss_G: 0.5809
[0/1000][62/782] Loss_D: 1.3690 Loss_G: 0.5802
[0/1000][63/782] Loss_D: 1.3503 Loss_G: 0.5838
[0/1000][64/782] Loss_D: 1.3185 Loss_G: 0.5853
[0/1000][65/782] Loss_D: 1.3141 Loss_G: 0.5827
[0/1000][66/782] Loss_D: 1.3376 Loss_G: 0.5778
[0/1000][67/7